<a href="https://colab.research.google.com/github/NazmusSaad/Medical-Assistant-LLM/blob/main/SImpleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_huggingface  # to download the weights for a pretrained LLM
!pip install chromadb  # open source vectorDB we can pass in chunks of our document
!pip install langchain_community  # helps in the process of embedding the query, searching the vectorDB, and passing to LLM
!pip install pypdf  # loading in pdf file
!pip install sentence-transformers  # can embedd documents using sentence transformers (giant BERT)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.1 MB/s 

In [2]:
!pip install -U langchain langchain-community langchain-core langchain-text-splitters transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 42.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.5
    Uninstalling huggingface-hub-0.33.5:
      Successfully uninstalled huggingface-hub-0.33.5
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  # or CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEndpoint  # use this instead of HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain

In [4]:
# Access secret token
from google.colab import userdata
API_TOKEN = userdata.get("HF_TOKEN")


In [5]:
from transformers import pipeline

# Load Zephyr or FLAN model locally
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", device=0)  # if on GPU
# pipe = pipeline("text-generation", model="google/flan-t5-xl")  # smaller fallback option

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:

from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipython-input-6-4043669678.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [7]:
!git clone https://github.com/gptandchill/rag_example
%cd rag_example

doc_loader = PyPDFLoader('sample_doc.pdf')
docs = doc_loader.load()

# split up doc into chunks of 1000 chars and no overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(docs)

Cloning into 'rag_example'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 88.00 KiB | 720.00 KiB/s, done.
/content/rag_example


In [8]:
query_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # NN that represents sentences as vector embeddings
vector_db = Chroma.from_documents(chunks, embedding=query_embedder, collection_name="docs") # the chunks will be stored efficiently in the vector db while the query embedder will be used to search for relevant chunks
top_k = 2
retriever = vector_db.as_retriever(search_kwargs={'k': top_k}) # algorithm to choose our top k chunks

/tmp/ipython-input-8-4091450965.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  query_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # NN that represents sentences as vector embeddings


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
chain = ConversationalRetrievalChain.from_llm(llm, retriever, return_source_documents=False)

In [ ]:
chats = []
while True:
  prompt = input('Prompt: ')
  if prompt == 'q':
    sys.exit()
  response = chain({'question': prompt, 'chat_history': chats})
  answer = response['answer']
  print(answer)
  helpful_answer = answer.find('Helpful Answer')
  if helpful_answer != -1:
    print(answer[helpful_answer:] + '\n')
  chats.append((prompt, answer))

Prompt: What film won the award for best picture


/tmp/ipython-input-10-2811035960.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({'question': prompt, 'chat_history': chats})
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

TheCreator–JayCooper, IanComley, AndrewRoberts,andNeilCorbouldGuardiansoftheGalaxyVol.3–StéphaneCeretti,AlexisWajsbrot,GuyWilliams,andTheoBialekMission:Impossible–DeadReckoningPartOne–AlexWuttke,SimoneCoco,Jeff Sutherland,andNeilCorbouldNapoleon–CharleyHenley, Luc-EwenMartin-Fenouillet,SimoneCoco,andNeilCorbouldGovernorsAwardsTheAcademyheldits14thannualGovernorsAwardsceremonyonJanuary9,2024,[d]whichwashostedbyJohnMulaney, duringwhichthefollowingawardswerepresented:[51][52][10]AcademyHonoraryAwardsAngelaBassett[53]MelBrooks[53]CarolLittleton[53]JeanHersholtHumanitarianAwardMichelleSatter[53]FilmswithmultiplenominationsandawardsFilmswithmultiplenominations[54][55]Nominations Film13Oppenheimer11PoorThings10KillersoftheFlowerMoon8 Barbie7 Maestro5 AmericanFictionAnatomyofaFallTheHoldoversTheZoneofInterest3 Napoleon2 TheCreatorMi

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

TheCreator–JayCooper, IanComley, AndrewRoberts,andNeilCorbouldGuardiansoftheGalaxyVol.3–StéphaneCeretti,AlexisWajsbrot,GuyWilliams,andTheoBialekMission:Impossible–DeadReckoningPartOne–AlexWuttke,SimoneCoco,Jeff Sutherland,andNeilCorbouldNapoleon–CharleyHenley, Luc-EwenMartin-Fenouillet,SimoneCoco,andNeilCorbouldGovernorsAwardsTheAcademyheldits14thannualGovernorsAwardsceremonyonJanuary9,2024,[d]whichwashostedbyJohnMulaney, duringwhichthefollowingawardswerepresented:[51][52][10]AcademyHonoraryAwardsAngelaBassett[53]MelBrooks[53]CarolLittleton[53]JeanHersholtHumanitarianAwardMichelleSatter[53]FilmswithmultiplenominationsandawardsFilmswithmultiplenominations[54][55]Nominations Film13Oppenheimer11PoorThings10KillersoftheFlowerMoon8 Barbie7 Maestro5 AmericanFictionAnatomyofaFallTheHoldoversTheZoneofInterest3 Napoleon2 TheCreatorMi

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

TheCreator–JayCooper, IanComley, AndrewRoberts,andNeilCorbouldGuardiansoftheGalaxyVol.3–StéphaneCeretti,AlexisWajsbrot,GuyWilliams,andTheoBialekMission:Impossible–DeadReckoningPartOne–AlexWuttke,SimoneCoco,Jeff Sutherland,andNeilCorbouldNapoleon–CharleyHenley, Luc-EwenMartin-Fenouillet,SimoneCoco,andNeilCorbouldGovernorsAwardsTheAcademyheldits14thannualGovernorsAwardsceremonyonJanuary9,2024,[d]whichwashostedbyJohnMulaney, duringwhichthefollowingawardswerepresented:[51][52][10]AcademyHonoraryAwardsAngelaBassett[53]MelBrooks[53]CarolLittleton[53]JeanHersholtHumanitarianAwardMichelleSatter[53]FilmswithmultiplenominationsandawardsFilmswithmultiplenominations[54][55]Nominations Film13Oppenheimer11PoorThings10KillersoftheFlowerMoon8 Barbie7 Maestro5 AmericanFictionAnatomyofaFallTheHoldoversTheZoneofInterest3 Napoleon2 TheCreatorMi